 **Configure environment**

In [13]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Token File,/home/fabric/.tokens.json
Bastion Host,bastion.fabric-testbed.net
Bastion Username,ashusri_0000051121
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Log File,/tmp/fablib/fablib.log
Log Level,INFO


**Node configuration**

In [14]:
slice_name="scherrer-original-" + fablib.get_bastion_username()

node_conf = [
 {'name': "mininet_node",   'cores': 4, 'ram': 16, 'disk': 100, 'image': 'default_ubuntu_22', 'packages': []}
]

exp_conf = {'cores': sum([ n['cores'] for n in node_conf]) }

**Reserve resources**

In [15]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

You don't have a slice named scherrer-original-ashusri_0000051121 yet.
Continue to the next step to make one.


We will select a random site that has sufficient resources for our experiment:

In [16]:
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_conf['cores']) ):
        break

fablib.show_site(site_name)

Name,TACC
State,Active
Address,"10100 Burnet Rd,Austin, TX 78758"
Location,"(0, 0)"
PTP Capable,False
Hosts,5
CPUs,10
Cores Available,422
Cores Capacity,640
Cores Allocated,218
RAM Available,1866


'<pandas.io.formats.style.Styler object at 0x7fc55508dc00>'

Then we will add the host

In [17]:
# this cell sets up the nodes
for n in node_conf:
    slice.add_node(name=n['name'], site=site_name, 
                   cores=n['cores'], 
                   ram=n['ram'], 
                   disk=n['disk'], 
                   image=n['image'])

The following cell submits our request to the FABRIC site. The output of this cell will update automatically as the status of our request changes.

* While it is being prepared, the “State” of the slice will appear as “Configuring”.
* When it is ready, the “State” of the slice will change to “StableOK”.

You may prefer to walk away and come back in a few minutes (for simple slices) or a few tens of minutes (for more complicated slices with many resources).

In [18]:
slice.submit()



Retry: 8, Time: 206 sec


ID,04638a2f-f307-45a5-9c6b-630525a34d6a
Name,scherrer-original-ashusri_0000051121
Lease Expiration (UTC),2024-04-01 19:37:51 +0000
Lease Start (UTC),2024-03-31 19:37:52 +0000
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
d968c56f-cf4d-4383-aec3-45cb94de8b35,mininet_node,4,16,100,default_ubuntu_22,qcow2,tacc-w5.fabric-testbed.net,TACC,ubuntu,129.114.110.85,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.85,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


'04638a2f-f307-45a5-9c6b-630525a34d6a'

In [19]:
slice.get_state()
slice.wait_ssh(progress=True)

Waiting for slice . Slice state: StableOK
Waiting for ssh in slice . ssh successful


True

**Log into resources**

In [20]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

,SSH command
Name,
mininet_node,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.85


**Install Mininet**

In [21]:
slice.get_node(name="mininet_node").execute('git clone https://github.com/mininet/mininet')
slice.get_node(name="mininet_node").execute('mininet/util/install.sh -a')
slice.get_node(name="mininet_node").execute('sudo mn --switch ovsbr --test pingall')

 Cloning into 'mininet'...
 Detected Linux distribution: Ubuntu 22.04 jammy amd64
sys.version_info(major=3, minor=10, micro=12, releaselevel='final', serial=0)
Detected Python (python3) version 3
Installing all packages except for -eix (doxypy, ivs, nox-classic)...
Install Mininet-compatible kernel if necessary
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1303 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy/universe amd64 Packages [14.1 MB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main Translation-en [233 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1616 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted Transla

('',
 '*** Creating network\n*** Adding controller\n*** Adding hosts:\nh1 h2 \n*** Adding switches:\ns1 \n*** Adding links:\n(h1, s1) (h2, s1) \n*** Configuring hosts\nh1 h2 \n*** Starting controller\nc0 \n*** Starting 1 switches\ns1 ...\n*** Waiting for switches to connect\ns1 \n*** Ping: testing ping reachability\nh1 -> h2 \nh2 -> h1 \n*** Results: 0% dropped (2/2 received)\n*** Stopping 1 controllers\nc0 \n*** Stopping 2 links\n..\n*** Stopping 1 switches\ns1 \n*** Stopping 2 hosts\nh1 h2 \n*** Done\ncompleted in 0.233 seconds\n')

**Clone original repo and install dependencies**

In [22]:
slice.get_node(name="mininet_node").execute('git clone https://github.com/ashutoshs25/imc22-bbr-fluid-model.git')
slice.get_node(name="mininet_node").execute('cd imc22-bbr-fluid-model; sudo bash ./install_dependencies.sh')
slice.get_node(name="mininet_node").execute('sudo apt-get -y install texlive-latex-extra')

 Cloning into 'imc22-bbr-fluid-model'...
 Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  cm-super-minimal dvisvgm fonts-droid-fallback fonts-lmodern fonts-noto-mono
  fonts-urw-base35 ghostscript libauthen-sasl-perl libclone-perl
  libdata-dump-perl libencode-locale-perl libfile-basedir-perl
  libfile-desktopentry-perl libfile-listing-perl libfile-mimeinfo-perl
  libfont-afm-perl libgs9 libgs9-common libhtml-form-perl libhtml-format-perl
  libhtml-parser-perl libhtml-tagset-perl libhtml-tree-perl
  libhttp-cookies-perl libhttp-daemon-perl libhttp-date-perl
  libhttp-message-perl libhttp-negotiate-perl libidn12 libijs-0.35
  libio-html-perl libio-socket-ssl-perl libio-stringy-perl
  libipc-system-simple-perl libjbig2dec0 libkpathsea6 liblwp-mediatypes-perl
  liblwp-protocol-https-perl libmailtools-perl libnet-dbus-perl
  libnet-http-perl libnet-smtp-ssl-perl libnet-ssleay-perl libpaper-utils
  libp

('Reading package lists...\nBuilding dependency tree...\nReading state information...\nThe following additional packages will be installed:\n  fonts-texgyre libapache-pom-java libcommons-logging-java\n  libcommons-parent-java libfontbox-java libpdfbox-java libruby3.0\n  preview-latex-style rake ruby ruby-net-telnet ruby-rubygems ruby-webrick\n  ruby-xmlrpc ruby3.0 rubygems-integration tex-gyre texlive-fonts-recommended\n  texlive-pictures texlive-plain-generic tipa tk tk8.6 unzip zip\nSuggested packages:\n  libavalon-framework-java libcommons-logging-java-doc\n  libexcalibur-logkit-java liblog4j1.2-java ri ruby-dev bundler\n  texlive-fonts-recommended-doc icc-profiles libfile-which-perl\n  libspreadsheet-parseexcel-perl texlive-latex-extra-doc dot2tex prerex\n  texlive-pictures-doc vprerex default-jre-headless tipa-doc\nThe following NEW packages will be installed:\n  fonts-texgyre libapache-pom-java libcommons-logging-java\n  libcommons-parent-java libfontbox-java libpdfbox-java libru

**Run BBR model checking experiment**

In [23]:
slice.get_node(name="mininet_node").execute('cd imc22-bbr-fluid-model;./run_model.py -c configs/bbr_model_checking.yaml')

configs/bbr_model_checking.yaml
{'senders': 1, 'link_capacity': 8.256274768824307, 'link_latency': 10, 'switch_buffer': 41, 'cc_combination': ['BBR'], 'source_latency_range': [5.0, 10.0], 'D': [31.4], 'd0': [5.7], 'qdisc': 'Drop-tail'}
0.0s
1.0s
2.0s
3.0s
4.0s
5.0s
6.0s
7.0s
8.0s
9.0s
10.0s
11.0s


("configs/bbr_model_checking.yaml\n{'senders': 1, 'link_capacity': 8.256274768824307, 'link_latency': 10, 'switch_buffer': 41, 'cc_combination': ['BBR'], 'source_latency_range': [5.0, 10.0], 'D': [31.4], 'd0': [5.7], 'qdisc': 'Drop-tail'}\n0.0s\n1.0s\n2.0s\n3.0s\n4.0s\n5.0s\n6.0s\n7.0s\n8.0s\n9.0s\n10.0s\n11.0s\n",
 '')

In [24]:
cmds_run_bbr_model_checking = '''
            cd imc22-bbr-fluid-model
            ./run_model.py -c configs/bbr_model_checking.yaml
            sudo ./run_experiments.py configs/bbr_model_checking.yaml
            ./plot_trace.py configs/bbr_model_checking.yaml
            ./plot.py configs/bbr_model_checking.yaml
            '''

slice.get_node(name="mininet_node").execute(cmds_run_bbr_model_checking)

configs/bbr_model_checking.yaml
{'senders': 1, 'link_capacity': 8.256274768824307, 'link_latency': 10, 'switch_buffer': 41, 'cc_combination': ['BBR'], 'source_latency_range': [5.0, 10.0], 'D': [31.4], 'd0': [5.7], 'qdisc': 'Drop-tail'}
0.0s
1.0s
2.0s
3.0s
4.0s
5.0s
6.0s
7.0s
8.0s
9.0s
10.0s
11.0s
{'send_duration': 12, 'truncate_front': 4, 'truncate_back': 2, 'plot_load_resolution': 0.01, 'pox_directory': '/home/ubuntu/pox/', 'senders': 1, 'link_capacity': 100, 'switch_buffer': 0.5, 'behavior_command': 'BBR-1', 'link_latency': 10, 'source_latency_range': [5.0, 10.0], 'use_red': False, 'name': '0'}
Run 1 / 1
Mininet Cleanup Start.
For Experiment:  results/bbr_model_checking/mininet_experiments/  doing run: 0
BBR-1
BBR-1
[31.2]
Starting host hDest
Starting queue measurement thread.
Invoking hosts...
Host #0 started.
Sending over:  1711915025.3127813
Closing queue measurement thread.
Initiating logparser: results/bbr_model_checking/mininet_experiments/results/12/1/100/0.5/BBR-1/2024-03-31-

("configs/bbr_model_checking.yaml\n{'senders': 1, 'link_capacity': 8.256274768824307, 'link_latency': 10, 'switch_buffer': 41, 'cc_combination': ['BBR'], 'source_latency_range': [5.0, 10.0], 'D': [31.4], 'd0': [5.7], 'qdisc': 'Drop-tail'}\n0.0s\n1.0s\n2.0s\n3.0s\n4.0s\n5.0s\n6.0s\n7.0s\n8.0s\n9.0s\n10.0s\n11.0s\n{'send_duration': 12, 'truncate_front': 4, 'truncate_back': 2, 'plot_load_resolution': 0.01, 'pox_directory': '/home/ubuntu/pox/', 'senders': 1, 'link_capacity': 100, 'switch_buffer': 0.5, 'behavior_command': 'BBR-1', 'link_latency': 10, 'source_latency_range': [5.0, 10.0], 'use_red': False, 'name': '0'}\nRun 1 / 1\nMininet Cleanup Start.\nFor Experiment:  results/bbr_model_checking/mininet_experiments/  doing run: 0\nBBR-1\nBBR-1\n[31.2]\nStarting host hDest\nStarting queue measurement thread.\nInvoking hosts...\nHost #0 started.\nSending over:  1711915025.3127813\nClosing queue measurement thread.\nInitiating logparser: results/bbr_model_checking/mininet_experiments/results/1

In [30]:
slice.get_node(name="mininet_node").download_file("/home/fabric/work/trace_original.png","/imc22-bbr-fluid-model/results/bbr_model_checking/mininet_experiments/results/12/1/100/0.5/BBR-1/2024-03-31--19-56-38/throughput_dist.png")





FileNotFoundError: [Errno 2] No such file

**Run 100 Mbps, 10 ms experiments**

In [37]:
cmds_run_scenario = '''
            cd imc22-bbr-fluid-model
            ./run_model.py -c configs/model_validation_100Mbps_5ms.yaml
            sudo ./run_experiments.py configs/model_validation_100Mbps_5ms.yaml
            ./plot_trace.py configs/model_validation_100Mbps_5ms.yaml
            ./plot.py configs/model_validation_100Mbps_5ms.yaml
            '''

slice.get_node(name="mininet_node").execute(cmds_run_scenario)

{'send_duration': 9, 'truncate_front': 4, 'truncate_back': 0, 'pox_directory': '/home/ubuntu/pox', 'senders': 10, 'link_capacity': 100, 'switch_buffer': 1.0, 'behavior_command': 'BBR-10', 'link_latency': 5, 'source_latency_range': [2.5, 5.0], 'use_red': True, 'name': '0'}
Run 1 / 2
Mininet Cleanup Start.
For Experiment:  results/model_validation_100Mbps_5ms/mininet_experiments/  doing run: 0
BBR-10
BBR-10
[15.6, 19.2, 18.8, 16.2, 17.4, 17.2, 18.2, 18.8, 15.4, 15.0]
Starting host hDest
Starting queue measurement thread.
Invoking hosts...
Host #0 started.
Host #1 started.
Host #2 started.
Host #3 started.
Host #4 started.
Host #5 started.
Host #6 started.
Host #7 started.
Host #8 started.
Host #9 started.
Sending over:  1711752903.3768206
Closing queue measurement thread.
Initiating logparser: results/model_validation_100Mbps_5ms/mininet_experiments/results/9/10/100/1.0/BBR-10/2024-03-29--22-54-42/
results/model_validation_100Mbps_5ms/mininet_experiments/results/9/10/100/1.0/BBR-10/2024-

("{'send_duration': 9, 'truncate_front': 4, 'truncate_back': 0, 'pox_directory': '/home/ubuntu/pox', 'senders': 10, 'link_capacity': 100, 'switch_buffer': 1.0, 'behavior_command': 'BBR-10', 'link_latency': 5, 'source_latency_range': [2.5, 5.0], 'use_red': True, 'name': '0'}\nRun 1 / 2\nMininet Cleanup Start.\nFor Experiment:  results/model_validation_100Mbps_5ms/mininet_experiments/  doing run: 0\nBBR-10\nBBR-10\n[15.6, 19.2, 18.8, 16.2, 17.4, 17.2, 18.2, 18.8, 15.4, 15.0]\nStarting host hDest\nStarting queue measurement thread.\nInvoking hosts...\nHost #0 started.\nHost #1 started.\nHost #2 started.\nHost #3 started.\nHost #4 started.\nHost #5 started.\nHost #6 started.\nHost #7 started.\nHost #8 started.\nHost #9 started.\nSending over:  1711752903.3768206\nClosing queue measurement thread.\nInitiating logparser: results/model_validation_100Mbps_5ms/mininet_experiments/results/9/10/100/1.0/BBR-10/2024-03-29--22-54-42/\nresults/model_validation_100Mbps_5ms/mininet_experiments/results/